# Table of Contents
 <p><div class="lev1 toc-item"><a href="#SQL2GEE" data-toc-modified-id="SQL2GEE-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>SQL2GEE</a></div><div class="lev2 toc-item"><a href="#SQL2json-response:" data-toc-modified-id="SQL2json-response:-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>SQL2json response:</a></div><div class="lev2 toc-item"><a href="#Feature-Collections;-data-tables" data-toc-modified-id="Feature-Collections;-data-tables-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Feature Collections; data tables</a></div><div class="lev2 toc-item"><a href="#Images" data-toc-modified-id="Images-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Images</a></div><div class="lev2 toc-item"><a href="#Image-Collections" data-toc-modified-id="Image-Collections-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Image Collections</a></div><div class="lev2 toc-item"><a href="#Map-display;-tiles-&amp;-geojsons" data-toc-modified-id="Map-display;-tiles-&amp;-geojsons-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Map display; tiles &amp; geojsons</a></div><div class="lev2 toc-item"><a href="#NRT-Image-generation" data-toc-modified-id="NRT-Image-generation-16"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>NRT Image generation</a></div>

In [35]:
import json
import ee
import folium
import requests
import numpy as np
ee.Initialize()

def tile_url(image, viz_params=None):
    """Create a target url for tiles for an image.
    e.g.
    im = ee.Image("LE7_TOA_1YEAR/" + year).select("B3","B2","B1")
    viz = {'opacity': 1, 'gain':3.5, 'bias':4, 'gamma':1.5}
    url = tile_url(image=im),viz_params=viz)
    """
    if viz_params:
        d = image.getMapId(viz_params)
    else:
        d = image.getMapId()
    base_url = 'https://earthengine.googleapis.com'
    url = (base_url + '/map/' + d['mapid'] + '/{z}/{x}/{y}?token=' + d['token'])
    return url



# SQL2GEE

## SQL2json response:

In [36]:
sql="""
SELECT a, sum(b) AS x, * 
FROM 'USDOS/LSIB/2013'
WHERE ST_INTERSECTS(ST_SetSRID(ST_GeomFromGeoJSON('{\"type\":\"Polygon\",\"coordinates\":[[[-5.273512601852417,42.81137220349083],[-5.273512601852417,42.811803118457306],[-5.272732079029083,42.811803118457306],[-5.272732079029083,42.81137220349083],[-5.273512601852417,42.81137220349083]]]}'), 4326), the_geom) 
and iso_num > 2 
and (iso_num < 10 or iso_num = 2) 
GROUP BY x LIMIT 1
"""
url='https://api.resourcewatch.org/v1/convert/sql2SQL'
payload={
    'sql':sql
}
r=requests.get(url, params=payload)
r.json()

{'data': {'attributes': {'jsonSql': {'from': "'USDOS/LSIB/2013'",
    'group': [{'type': 'literal', 'value': 'x'}],
    'limit': 1,
    'select': [{'alias': None, 'type': 'literal', 'value': 'a'},
     {'alias': 'x',
      'arguments': [{'type': 'literal', 'value': 'b'}],
      'type': 'function',
      'value': 'sum'},
     {'alias': None, 'type': 'wildcard', 'value': '*'}],
    'where': {'left': {'left': {'alias': None,
       'arguments': [{'alias': None,
         'arguments': [{'alias': None,
           'arguments': [{'type': 'string',
             'value': '\'{"type":"Polygon","coordinates":[[[-5.273512601852417,42.81137220349083],[-5.273512601852417,42.811803118457306],[-5.272732079029083,42.811803118457306],[-5.272732079029083,42.81137220349083],[-5.273512601852417,42.81137220349083]]]}\''}],
           'type': 'function',
           'value': 'ST_GeomFromGeoJSON'},
          {'type': 'number', 'value': 4326}],
         'type': 'function',
         'value': 'ST_SetSRID'},
       

In [37]:
r.json()['data']['attributes']['jsonSql']['select']

[{'alias': None, 'type': 'literal', 'value': 'a'},
 {'alias': 'x',
  'arguments': [{'type': 'literal', 'value': 'b'}],
  'type': 'function',
  'value': 'sum'},
 {'alias': None, 'type': 'wildcard', 'value': '*'}]

In [38]:
r.json()['data']['attributes']['jsonSql']['where']['type']

'conditional'

In [39]:
r.json()['data']['attributes']['jsonSql']['where']['right']

{'type': 'bracket',
 'value': {'left': {'left': {'type': 'literal', 'value': 'iso_num'},
   'right': {'type': 'number', 'value': 10},
   'type': 'operator',
   'value': '<'},
  'right': {'left': {'type': 'literal', 'value': 'iso_num'},
   'right': {'type': 'number', 'value': 2},
   'type': 'operator',
   'value': '='},
  'type': 'conditional',
  'value': 'or'}}

Once we get the sql that we want to perform we will need to build it up for GEE.

In order to check the dataset type we can use `ee.data.getInfo(id, callback)`.   
If we are dealing with Fusion table dataset, the asset-id will start with `ft:` and will not run using that function.

In [40]:
#      [FeatureCollection,fusion table id(ft:<id>), image collection, image]


assets = ['USDOS/LSIB/2013','1tdSwUL7MVpOauSgRzqVTOwdfy17KDbw-1d9omPw','NOAA/DMSP-OLS/NIGHTTIME_LIGHTS','NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012','blalbal']
for f in assets:
    t=ee.data.getInfo(f)
    if t!=None:
        
        print('ee data type: ',t['type'])
        print('keys: ',t.keys())
        print('******************************')
        print('keys: ',t)
    else:
        print(t)
    print('-----------------')

ee data type:  FeatureCollection
keys:  dict_keys(['type', 'columns', 'id', 'version', 'properties'])
******************************
keys:  {'type': 'FeatureCollection', 'columns': {'cc': 'String', 'iso_alpha2': 'String', 'iso_alpha3': 'String', 'iso_num': 'Float', 'name': 'String', 'region': 'String', 'system:index': 'String', 'tld': 'String'}, 'id': 'USDOS/LSIB/2013', 'version': 1509487966253039, 'properties': {'date_range': [1362700800000.0, 1362700800000.0], 'period': 0.0, 'thumb': 'https://mw1.google.com/ges/dd/images/LSIB_thumb.png', 'description': '<p>\nThe United States Office of the Geographer provides the Large Scale\nInternational Boundary Lines, an international boundary-line vector file. The\ninterior lines reflect U.S. government policy and thus do not necessarily\ncorrespond to de facto control.  The 1:250,000 scale World Vector Shoreline\nlines are generally accurate from between several hundred meters to over a\nkilometer. Each feature is the polygonal area enclosed by

None
-----------------


## Feature Collections; data tables

In [41]:
ee.Filter.rangeContains

<function ee.apifunction.ApiFunction.importApi.<locals>.MakeBoundFunction.<locals>.<lambda>>

In [42]:
## myFeature = ee.FeatureCollection('USDOS/LSIB/2013')
myFeature = ee.FeatureCollection('ft:1_RZgjlcqixp-L9hyS6NYGqLaKOlnhSC35AB5M5Ll') 
## Dictionay respone object example
myFeature.limit(1).getInfo().keys()

dict_keys(['type', 'columns', 'properties', 'features'])

Query structure for tables:
```sql
SELECT <wildcard(*)>|<columns>|<function<column>>|**<agg_functions<columns>>
FROM <asset_id>
WHERE (<filter_condictions> <conditional> <filter_condictions>)
**GROUP BY <columns>
ORDER BY <columns>
LIMIT <output_nRows>
```  
 
The order for GEE will be `FCollection.<filters>.<functions>.<sorts>.limit(n).getInfo()`  
We should allways give back the size of the computed collection

We can paginate the colection with `this.toList(count, offset)`

#### To be able to check all columns on a table

In [43]:
myFeature.limit(0).getInfo()['columns'].keys()

dict_keys(['AREA', 'DAYCLASS', 'MODE', 'PATH', 'PERIMETER', 'PR', 'PR_', 'PR_ID', 'RINGS_NOK', 'RINGS_OK', 'ROW', 'SEQUENCE', 'WRSPR'])

### SELECT

In [44]:
# WILDCARD
print(myFeature.limit(1).getInfo()['features'][0]['properties'])
# COLUMN SELECTION
print(myFeature.select('DAYCLASS', 'MODE').limit(1).getInfo()['features'][0]['properties'])

{'AREA': 15.74326, 'DAYCLASS': 1.0, 'MODE': 'D', 'PATH': 13.0, 'PERIMETER': 26.98611, 'PR': 13001.0, 'PR_': 1.0, 'PR_ID': 1.0, 'RINGS_NOK': 0.0, 'RINGS_OK': 1.0, 'ROW': 1.0, 'SEQUENCE': 2233.0, 'WRSPR': 13001.0}
{'DAYCLASS': 1.0, 'MODE': 'D'}


In [45]:
print(myFeature.select(['DAYCLASS', 'MODE']).limit(1).getInfo()['features'][0]['properties'])

{'DAYCLASS': 1.0, 'MODE': 'D'}


In [46]:
# FUNCTION


AGG - FUNCTIONS

In [47]:
# WITHOUT GROUPBY
## array, count, count_distinct, first, histogram, max, mean, min, product, sample_sd, sample_var, stats, sum, total_sd, total_var
print(myFeature.select('DAYCLASS','PR_ID').aggregate_min('DAYCLASS').getInfo())
print(myFeature.select('DAYCLASS','PR_ID').aggregate_sum('DAYCLASS').getInfo())

1.0
246140.0


### FROM

we will get asset id

### WHERE

In [48]:
# Conditional with this.filter(ee.Filter.and(...)|ee.Filter.or(...))
myFeature.filter(ee.Filter([ee.Filter.neq('DAYCLASS', 5)])).limit(1).getInfo()

{'columns': {'AREA': 'Number',
  'DAYCLASS': 'Number',
  'MODE': 'String',
  'PATH': 'Number',
  'PERIMETER': 'Number',
  'PR': 'Number',
  'PR_': 'Number',
  'PR_ID': 'Number',
  'RINGS_NOK': 'Number',
  'RINGS_OK': 'Number',
  'ROW': 'Number',
  'SEQUENCE': 'Number',
  'WRSPR': 'Number'},
 'features': [{'geometry': {'coordinates': [[[-10.803414, 80.9888],
      [-11.410384900000002, 81.203407],
      [-11.681594000000002, 81.29929790000001],
      [-13.2360839, 81.848918],
      [-13.254700000000001, 81.85550000000002],
      [-13.487837899999997, 81.83563],
      [-13.716189900000002, 81.816168],
      [-16.4464359, 81.58347300000001],
      [-16.943174000000003, 81.541136],
      [-21.730125, 81.133151],
      [-21.78798, 81.12822],
      [-22.005462, 81.109684],
      [-22.215300000000006, 81.0918],
      [-22.194578, 81.086242],
      [-21.941443000000007, 81.01834399999998],
      [-21.364428000000007, 80.8635709],
      [-20.5893919, 80.6556839],
      [-19.05658, 80.244539],
 

In [49]:
# Filter with this.filter(ee.Filter.<>(...))

In [50]:
myFeature.filter(ee.Filter.Or([ee.Filter.neq('DAYCLASS', 5),ee.Filter.gt('DAYCLASS', 9)])).limit(1).getInfo()

{'columns': {'AREA': 'Number',
  'DAYCLASS': 'Number',
  'MODE': 'String',
  'PATH': 'Number',
  'PERIMETER': 'Number',
  'PR': 'Number',
  'PR_': 'Number',
  'PR_ID': 'Number',
  'RINGS_NOK': 'Number',
  'RINGS_OK': 'Number',
  'ROW': 'Number',
  'SEQUENCE': 'Number',
  'WRSPR': 'Number'},
 'features': [{'geometry': {'coordinates': [[[-10.803414, 80.9888],
      [-11.410384900000002, 81.203407],
      [-11.681594000000002, 81.29929790000001],
      [-13.2360839, 81.848918],
      [-13.254700000000001, 81.85550000000002],
      [-13.487837899999997, 81.83563],
      [-13.716189900000002, 81.816168],
      [-16.4464359, 81.58347300000001],
      [-16.943174000000003, 81.541136],
      [-21.730125, 81.133151],
      [-21.78798, 81.12822],
      [-22.005462, 81.109684],
      [-22.215300000000006, 81.0918],
      [-22.194578, 81.086242],
      [-21.941443000000007, 81.01834399999998],
      [-21.364428000000007, 80.8635709],
      [-20.5893919, 80.6556839],
      [-19.05658, 80.244539],
 

###  GROUP BY + agg column 

In [51]:
# GROUPING BY
## array, count, count_distinct, first, histogram, max, mean, min, product, sample_sd, sample_var, stats, sum, total_sd, total_var ...

myReducer= {
     'reducer': ee.Reducer.count().combine(ee.Reducer.sum(), outputPrefix='', sharedInputs=True).repeat(2).group(**{
            'groupField': 2,
            'groupName': 'PR_ID',
    }),
    'selectors': ['DAYCLASS','PR_ID','MODE']
}
myFeature.select('DAYCLASS','PR_ID','MODE').reduceColumns(**myReducer).getInfo()

{'groups': [{'PR_ID': 'D',
   'count': [28892, 28892],
   'sum': [246140.0, 50513198.0]}]}

### ORDER BY

In [ ]:
#this.sort(property, ascending)


### LIMIT

In [ ]:
#this.limit(number)


## Images

In [52]:
myImg = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012') 

In [53]:
print(myImg.getInfo().keys())

dict_keys(['type', 'bands', 'version', 'id', 'properties'])


Capabilities:
* Fix a region (x,y axes) and be able to filter and calculate zonal statistics within temporal axes or date available

* Fix a date/category (z axes) and be able to work as if it is a simple image

It will depend on the asset that we will be able to filter over each image in the collection or the values on the images bands.

Query structure for Image collection which its output should be a table with n columns and rows:
```sql  
SELECT <bands>|<functions<bands>>|**<agg_functions<bands>>
FROM <asset_id>
WHERE (<filter_condictions> <conditional> <filter_condictions>)
**GROUP BY <columns>
ORDER BY <columns>
LIMIT <output_nRows>
```  

Output:  
 ```
 {"data":[
     {"col_a": 12,"col_b":"asdas"},
     {"col_a": 12,"col_b":"asdas"},
     ...
     ]
 }
 ```
 
| column a  |  ... | column n |
|---|---|---|
| ..  | ..  |  .. |

 
The order for GEE will be `Image.<filters>.<functions>.<imageReducers>.getInfo().<sorts>.limit()`  
We should allways give back the size `this.size()` of the computed collection

We will need to give back in the metadata this: 
size, metadata for the image, bands available and the data type stored in there.

***Images doesn't have `this.limit()` or `this.sort()` so we need to limit and sort after the reduction** 

## Image Collections

In [54]:
myImCol = ee.ImageCollection('NASA/NEX-GDDP') 
## Dictionay respone object example
print(myImCol.limit(1).getInfo().keys())

print(myImCol.limit(1).getInfo()['bands'])
print(myImCol.limit(1).getInfo()['properties'].keys())
print(myImCol.limit(1).getInfo()['features'][0].keys())
print(myImCol.limit(1).getInfo()['features'][0]['properties'].keys())

dict_keys(['type', 'bands', 'id', 'version', 'properties', 'features'])
[]
dict_keys(['date_range', 'period', 'system:visualization_0_bands', 'system:is_global', 'description', 'source_tags', 'provider_url', 'title', 'tags', 'product_tags', 'provider', 'system:visualization_0_name'])
dict_keys(['type', 'bands', 'version', 'id', 'properties'])
dict_keys(['system:time_start', 'month', 'scenario', 'year', 'system:footprint', 'model', 'system:asset_size', 'day', 'system:index'])


Image collections are similar to ndarrays with 4 dimensions. (x,y,(bands),z)
Where z could be a temporal dimension or a categorical one.
For the minimum viable product we intend to build capabilities under the temporal dimension, but probably it will be possible to extend it to diferent dimensions.

Capabilities:
* Fix a region (x,y axes) and be able to filter and calculate zonal statistics within temporal axes or date available

* Fix a date/category (z axes) and be able to work as if it is a simple image

It will depend on the asset that we will be able to filter over each image in the collection or the values on the images bands.

Query structure for Image collection which its output should be a table with n columns and rows:
```sql  
SELECT <columns/Image & bands>|<functions<column>>|**<agg_functions<columns>>
FROM <asset_id>
WHERE (<filter_condictions> <conditional> <filter_condictions>)
**GROUP BY <columns>
ORDER BY <columns>
LIMIT <output_nRows>
```  
 Output:  
 ```
 {"data":[
     {"col_a": 12,"col_b":"asdas"},
     {"col_a": 12,"col_b":"asdas"},
     ...
     ]
 }
 ```
 
| column a  |  ... | column n |
|---|---|---|
| ..  | ..  |  .. |

 
The order for GEE will be `imageCollection.<filters>.<functions>.<sorts>.<imageReducers>.limit(n).getInfo()`  
We should allways give back the size `this.size()` of the computed collection

We will need to give back in the metadata this: 
size, metadata/columns for the collection, bands available and the data type stored in there.

In [55]:
print(myImCol.limit(1).getInfo()['properties']['date_range'])
print(myImCol.limit(1).getInfo()['properties']['period'])
print([band['id'] for band in myImCol.limit(1).getInfo()['features'][0]['bands']])
print(myImCol.limit(1).getInfo()['features'][0]['properties'].keys())

[-631152000000.0, 4102358400000.0]
0.0
['pr', 'tasmin', 'tasmax']
dict_keys(['system:time_start', 'month', 'scenario', 'year', 'system:footprint', 'model', 'system:asset_size', 'day', 'system:index'])


### SELECT

In [56]:
# WILDCARD will get the bands, and each image metadata
print(myImCol.limit(1).getInfo()['features'][0]['properties'].keys())
print([band['id'] for band in myImCol.limit(1).getInfo()['features'][0]['bands']])

# COLUMN & BAND SELECTION
print(myImCol.select('pr').limit(1).getInfo()['features'][0]['bands']) ## Selects a band


dict_keys(['system:time_start', 'month', 'scenario', 'year', 'system:footprint', 'model', 'system:asset_size', 'day', 'system:index'])
['pr', 'tasmin', 'tasmax']
[{'id': 'pr', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1441, 720], 'crs': 'EPSG:4326', 'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0]}]


In [15]:
print(myImCol.limit(1).toList(3).getInfo()) ## Selects columns

[{'type': 'Image', 'bands': [{'id': 'pr', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1441, 720], 'crs': 'EPSG:4326', 'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0]}, {'id': 'tasmin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1441, 720], 'crs': 'EPSG:4326', 'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0]}, {'id': 'tasmax', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1441, 720], 'crs': 'EPSG:4326', 'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0]}], 'version': 1494551250391000, 'id': 'NASA/NEX-GDDP/historical_ACCESS1-0_19500101', 'properties': {'system:time_start': -631152000000.0, 'month': 1.0, 'scenario': 'historical', 'year': 1950.0, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[-180.0, -90.0], [180.0, -90.0], [180.0, 90.0], [-180.0, 90.0], [-180.0, -90.0]]}, 'model': 'ACCESS1-0', 'system:asset_size': 10740266, 'day': 1.0, 'system:index': 'historical_ACCESS1-0_1

In [ ]:
# Agg Functions

### FROM 

img collection asset id

### WHERE

In [16]:
# Where filter by date and geometry
geometry =ee.Geometry.Polygon(
         [[[-114.8291015625, 34.27083595164997],
          [-115.2685546875, 32.65787573695528],
          [-113.8623046875, 33.79740876757251]]]);



filtered=myImCol.filter(ee.Filter([ee.Filter.eq('scenario','historical'),
                                   ee.Filter.date('1996-01-01','1997-01-01')])).filterBounds(geometry).select('pr').sort('system:time_start')
print(filtered.size().getInfo())

7672


Issues:
 * Region size under reduction, i have to check if there is a way for GEE to split the computations chunks to reduce computing time
 * Check its capabilities with Arrays 

In [29]:
# Compute the mean precipitation in the region in each image. this redice the x/y axis acros a z axis for the nbands available
def ComputeMean(img):
    reducer={
        'reducer': ee.Reducer.sum(), 
        'geometry': geometry, 
        'bestEffort': True,
        'tileScale':4,
        'scale': 90
    }
    reduction = img.reduceRegion(**reducer)
    return ee.Feature(None, {
        'result': reduction,
        'system:time_start': img.get('system:time_start')
    })

chart_data = filtered.map(ComputeMean)
print(chart_data.toList(8000).getInfo())

ee.Image({
  "type": "ArgumentRef",
  "value": null
})
ee.Image({
  "type": "ArgumentRef",
  "value": "_MAPPING_VAR_0_0"
})


KeyboardInterrupt: 

this.toArray() allow band parallel computation and reduction in a single image. For temporal ranges we will need to use the above reducer

#### Try to reduce on the z-axe image stacks. Lets check differences between reduce/reduceImage/reduceColumns in image collections

In [281]:
# Reduce
filtered=myImCol.filter(ee.Filter([ee.Filter.eq('scenario','historical'),
                                   ee.Filter.date('1996-01-01','1997-01-01')])).filterBounds(geometry).select('pr').sort('system:time_start')
print(filtered.size().getInfo())
myReducer= {
     'reducer': ee.Reducer.sum()
}
image_test = filtered.reduce(**myReducer)
print(image_test.getInfo())


tilesa = tile_url(image_test.clip(geometry),{'opacity': 1})
# map.add_tile_layer(tiles=tiles_lc, attr="LandCover")
map = folium.Map(tiles='Mapbox Bright', location=[34.27083595164997,-114.8291015625], zoom_start = 7, width=300,height=300)
map.add_tile_layer(tiles=tilesa, attr="Reduce")
map

7672
{'type': 'Image', 'bands': [{'id': 'pr_sum', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}]}


In [152]:
# ReduceToImage
filtered=myImCol.filter(ee.Filter([ee.Filter.eq('scenario','historical'),ee.Filter.date('1996-01-01','1997-01-01')])).filterBounds(geometry).sort('system:time_start')
print(filtered.size().getInfo())
myReducer= {
     'reducer': ee.Reducer.sum(),
    'properties':['month']
}
image_test = filtered.select('pr').reduceToImage(**myReducer)
print(image_test.getInfo())


tilesa = tile_url(image_test.clip(geometry),{'opacity': 1})
# map.add_tile_layer(tiles=tiles_lc, attr="LandCover")
map = folium.Map(tiles='Mapbox Bright', location=[34.27083595164997,-114.8291015625], zoom_start = 5, width=300,height=300)
map.add_tile_layer(tiles=tilesa, attr="Reduce")
map

7672
{'type': 'Image', 'bands': [{'id': 'sum', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}]}


In [17]:
# Filter example
filtered=myImCol.filter(ee.Filter([ee.Filter.eq('scenario','historical'),
                                   ee.Filter.date('1990-01-01','2000-01-01')])).select('pr').sort('system:time_start')
print(filtered.size().getInfo())

# Reduce Column as expected only operates in the properties; lets use it for cross product calculation
# Reducer 
# 'month', 'scenario', 'year', 'model'
myReducer= {
      'reducer': ee.Reducer.count().repeat(1).group(**{
            'groupField': 0,
            'groupName': 'model',
    }).group(**{
            'groupField': 0,
            'groupName': 'month',
    }).group(**{
            'groupField': 0,
            'groupName': 'year',
    }),
    'selectors': ['year','month','model','year']
}

76664


In [18]:
"""
For filter generation the input shoud be the where tree generated by sql2json
"""

def filterGen(data, parents=[]):
    _filters = {
            '<': ee.Filter.lt,
            '<=': ee.Filter.lte,
            '>': ee.Filter.gt,
            '>=': ee.Filter.gte,
            '<>': ee.Filter.neq,
            '=': ee.Filter.eq,
            '!=': ee.Filter.neq,
            'bedate': ee.Filter.date,
            'between': ee.Filter.rangeContains,
            'like': ee.Filter.eq,
            '%like%': ee.Filter.stringContains,
            '%like': ee.Filter.stringEndsWith,
            'like%': ee.Filter.stringStartsWith
            
        }
    _comparisons = {
            'and': ee.Filter.And,
            'or': ee.Filter.Or
        }
    left = None
    right = None
    result =[]
    dparents=list(parents)
    if 'type' in [*data] and data['type']=='conditional': ########---------------------------- Leaf with groups l and r:
        left = data['left']
        right = data['right']
        dparents=[data['value']]
    elif 'type' in [*data] and data['type']=='operator': ########------------------------------------------- Latests leaf we will want to return  
        return _filters[data['value']](data['left']['value'],data['right']['value'])
    
    if left and right: ########-------------------------------------- leaf group iteration
        #for l in left:
        partialL=filterGen(left, dparents)
        #for r in right:
        partialR=filterGen(right, dparents)
        result=_comparisons[dparents[0]](partialR,partialL)
        
    return result ########----------------------------------- Return result in:[[ee.Filter.eq('month','historical'),ee.Filter.eq('model','historical'),...],...]   

#"""
#For reducer generation the input shoud be the groupby and select function generated by sql2json
#"""    
##def reducerGen():
#    """
#    Operation + groupby columns
#    """
#    reducTyDict={
#        "":
#    }
#    reducDict={
#        "":
#    }

In [31]:
filtered=myImCol.filter(ee.Filter([ee.Filter.eq('scenario','historical'), ee.Filter.date('1996-01-01','1997-01-01')])).filterBounds(geometry).sort('system:time_start')
def myPrint(data, parents=[]):
    keys=[*data]
    groups = None
    result =[]
    dparents=list(parents)
    if 'groups' in keys: ########---------------------------- Leaf with groups:
        groups = data['groups']
        if (len(keys) > 1): ########------------------------- If groups not alone:
            keys.remove('groups')
            dparents.append(ee.Filter.eq(keys[0],data[keys[0]]))
    else: ########------------------------------------------- Latests leaf we will want to return  
        keys.remove('count')
        return [ee.Filter.eq(keys[0],data[keys[0]]),*dparents]
    
    if groups: ########-------------------------------------- leaf group iteration
        for group in groups:
            partialR=myPrint(group, dparents)
            ##----------------------------------------------- to keep it in a 2d array
            if isinstance(partialR[0], list):
                result.extend(partialR)
            else:
                result.append(partialR)
    return result ########----------------------------------- Return result in:[[ee.Filter.eq('month','historical'),ee.Filter.eq('model','historical'),...],...] 

def ComputeMean(img):
    reducer={
        'reducer': ee.Reducer.mean(), 
        'geometry': geometry, 
        'bestEffort': True,
        'tileScale':4,
        'scale': 90
    }
    reduction = img.reduceRegion(**reducer)
    return ee.Feature(None, {
        'result': reduction,
        'system:time_start': img.get('system:time_start')
    })
#def imageReduce(subCollection):
#    myReducer={
#      'reducer': ee.Reducer.mean(),
#      'parallelScale':4
#    }
#    print(subCollection.evaluate())
#    print('n')
#    return subCollection.reduce(**myReducer)    
    
def collection_reducer(collection):
    myReducer= {
      'reducer': ee.Reducer.count().repeat(1).group(**{
            'groupField': 0,
            'groupName': 'year',
    }).group(**{
            'groupField': 0,
            'groupName': 'month',
    }),
    'selectors': ['month','year','model']
    }
    myReducer2={
      'reducer': ee.Reducer.mean(),
      'parallelScale':4
    }
    crossP=collection.reduceColumns(**myReducer)
    mysubsets=myPrint(crossP.getInfo())
    myList=ee.List([collection.filter(ee.Filter(filters)).reduce(**myReducer2).set()  for filters in mysubsets])
    return ee.ImageCollection(myList)

In [32]:
imgn=collection_reducer(filtered)

In [33]:
imgn.getInfo()

{'bands': [],
 'features': [{'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
     'data_type': {'precision': 'float', 'type': 'PixelType'},
     'id': 'pr_mean'},
    {'crs': 'EPSG:4326',
     'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
     'data_type': {'precision': 'float', 'type': 'PixelType'},
     'id': 'tasmin_mean'},
    {'crs': 'EPSG:4326',
     'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
     'data_type': {'precision': 'float', 'type': 'PixelType'},
     'id': 'tasmax_mean'}],
   'properties': {'system:index': '0'},
   'type': 'Image'},
  {'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
     'data_type': {'precision': 'float', 'type': 'PixelType'},
     'id': 'pr_mean'},
    {'crs': 'EPSG:4326',
     'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
     'data_type': {'precision': 'float', 'type': 'PixelType'},
     'id': 'tasmin_mean'},
    {'crs': 'EPSG:4326',
     'crs_transform': [1.0, 0.0,

In [34]:
# Compute the mean precipitation in the region in each image. this redice the x/y axis acros a z axis for the nbands available
timeSeries=imgn.map(ComputeMean).toList(800).getInfo()
print(timeSeries)

[{'type': 'Feature', 'geometry': None, 'id': '0', 'properties': {'result': {'pr_mean': 2.418801815260752e-06, 'tasmax_mean': 293.35888747809923, 'tasmin_mean': 278.2260322798954}}}, {'type': 'Feature', 'geometry': None, 'id': '1', 'properties': {'result': {'pr_mean': 6.321651352080455e-06, 'tasmax_mean': 295.5712528670049, 'tasmin_mean': 280.0673959194391}}}, {'type': 'Feature', 'geometry': None, 'id': '2', 'properties': {'result': {'pr_mean': 2.77826147372122e-06, 'tasmax_mean': 297.89602939536934, 'tasmin_mean': 281.79534290900926}}}, {'type': 'Feature', 'geometry': None, 'id': '3', 'properties': {'result': {'pr_mean': 7.799395538504405e-07, 'tasmax_mean': 301.94299686527506, 'tasmin_mean': 284.49844900298194}}}, {'type': 'Feature', 'geometry': None, 'id': '4', 'properties': {'result': {'pr_mean': 3.349269122031681e-10, 'tasmax_mean': 306.50344557937035, 'tasmin_mean': 288.5175437028879}}}, {'type': 'Feature', 'geometry': None, 'id': '5', 'properties': {'result': {'pr_mean': 4.605523

In [21]:
img=collection_reducer(filtered)
#print(img.map(ComputeMean).getInfo())
tilesa = tile_url(ee.Image(img.first()),{'min':0,'max':0.00001,'opacity': 1})
# map.add_tile_layer(tiles=tiles_lc, attr="LandCover")
map = folium.Map(tiles='Mapbox Bright', location=[33.29083595164997,-113.8291015625], zoom_start = 7, width=800,height=300)
map.add_tile_layer(tiles=tilesa, attr="Reduce")
map

In [228]:
farray.arrayReduce(**reducer).arrayProject([imageAxis]).getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   'data_type': {'dimensions': 1, 'precision': 'double', 'type': 'PixelType'},
   'id': 'array'}],
 'type': 'Image'}

In [229]:
filtered.max().getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   'data_type': {'precision': 'float', 'type': 'PixelType'},
   'id': 'pr'}],
 'type': 'Image'}

In [290]:
###DateTime management in GEE
print(ee.DateRange(ee.Date('1996-01-01'), ee.Date('1997-01-01')).getInfo())
rangeD=ee.DateRange(ee.Date('1996-01-01'), ee.Date('1997-01-01')).getInfo()['dates']
print(ee.Date.fromYMD(1990, 5, 10).getInfo())
angeD=myImCol.limit(1).getInfo()['properties']['date_range']
print(ee.DateRange(rangeD[0],rangeD[1]).getInfo())
print(ee.Date(rangeD[0]).format('yyyy-mm-dd').getInfo())
print(ee.Date(rangeD[1]).format('yyyy-mm-dd').getInfo())
print(myImCol.size().getInfo())

{'type': 'DateRange', 'dates': [820454400000, 852076800000]}
{'type': 'Date', 'value': 642297600000}
{'type': 'DateRange', 'dates': [820454400000, 852076800000]}
1996-00-01
1997-00-01
1882359


## Map display; tiles & geojsons

### Feature Collection

Depends on what we want we can get the geojson associated or the tiles

* Tiles: We will need to pass asset_id and the styles (in the future maybe a function to be applied to all polygons?) this.getMap(visParams, callback) or this.style().getMap()

* Geojson: We will need to pass asset_id and the styles (in the future maybe a function to be applied to all polygons?) this.toList()

### Image

We will need to pass asset_id and the styles (in the future maybe a function to be applied to all polygons?)

### Image Collection

we need to composite the image Collection first so there is a set of params that we will want to pass: filter columns, time range and the composite method (first image by default).

We will allow to pass params or a sql to be able to reduce/select an image and then the colorizing methodology will be the same as the one used for images

In [ ]:
#ee.Image(imgC.first())

## NRT Image generation

The process can be summarize here:
* Image upload to CloudStorage
* move the image into the Collection while adding some basic Metadata

1.- We create our collection:

>```bash
earthengine create collection users/username/collection_id
```
We set it to public access:
```bash
earthengine acl set public  users/username/collection_id
```
And we add it's metadata:
```bash
earthengine asset set --time_start YYYY-MM-DDThh:mm:ss --time_end YYYY-MM-DDThh:mm:ss -p name=value users/username/collection_id
``` 

2.- On our script once we have manipulate/create our tif, we will upload it to our Google cloud storage bucket:
>```bash
gsutil mb gs://[YOUR-BUCKET-NAME]
gsutil defacl set public-read gs://[YOUR-BUCKET-NAME]
gsutil cp users/mypath/myraster.tif gs://my-awesome-bucket
``` 

3.- Now that our image is on cloud storage we will move it to our collection
mean (the default), sample, mode, min, or max
>```bash
earthengine upload image --asset_id=users/username/collection_id/asset_id --nodata_value=<nodata> --pyramiding_policy=sample gs://bucket/image.tif
``` 
And finally we will set the properties for that image
e
```bash
earthengine asset set --time_start YYYY-MM-DDThh:mm:ss -p name=value users/username/collection_id/asset_id
``` 

For more info:
* https://github.com/Vizzuality/data_sci_tutorials/blob/master/work/INGESTING_EARTH_ENGINE_ASSETS.md#earth-engine-data-ingress
* https://developers.google.com/earth-engine/command_line
* https://cloud.google.com/python/getting-started/using-cloud-storage